In [1]:
import pandas as pd
import datetime
import numpy as np
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler

In [2]:
df = pd.read_csv('dealer_prioritization_upstream_open_new.csv')

In [3]:
def pad(hist, length=90):
    l = len(hist)
    return [0] * (length - l) + hist

In [4]:
df.head()

,dow,purchase_date,purchases,purchased_flag,buyer_cluster,open_purchases_daily_30,open_purchases_daily_pct_30,open_purchases_daily_180,open_purchases_daily_pct_180,avg_time_between_open_purchases,...,cars_moved_off_retail_90,cluster_G,cluster_NA,cluster_NG,cluster_O,cluster_P+,cluster_P-,cluster_SC,cluster_Super,buyer_id_masked
0,4,2020-01-02,0,0,SC,0,0.00,2,0.29,28,...,226,0,0,0,0,0,0,1,0,0
1,4,2020-01-02,0,0,NG,0,0.00,2,0.15,90,...,66,0,0,1,0,0,0,0,0,1
2,4,2020-01-02,0,0,O,2,0.22,4,0.08,4,...,0,0,0,0,1,0,0,0,0,2
3,4,2020-01-02,0,0,NaN,0,0.00,0,0.00,90,...,247,0,1,0,0,0,0,0,0,3
4,4,2020-01-02,1,1,O,1,0.11,13,0.19,5,...,0,0,0,0,1,0,0,0,0,4


In [5]:
df['purchase_date'] = pd.to_datetime(df['purchase_date']) 

In [6]:
df['purchases']=df['purchases'].fillna(0)

In [7]:
scaler = MinMaxScaler()
df['purchases'] = scaler.fit_transform(df[['purchases']])

In [8]:
res = np.zeros((df.shape[0], 90))
for id in tqdm(df['buyer_id_masked'].unique()):
  dt = df[df['buyer_id_masked'] == id]
  for date in dt['purchase_date']:
    index = dt.loc[dt['purchase_date'] == date].index[0]
    hist = dt[((date - pd.DateOffset(days=90))<=dt['purchase_date']) & (dt['purchase_date'] < date)]['purchases'].tolist()
    hist = pad(hist)
    res[index,:] = hist

100%|██████████| 2423/2423 [03:30<00:00, 11.50it/s]


In [9]:
np.save('history.npy', res)

In [25]:
df2 = pd.DataFrame(res)

In [128]:
df2['label'] = df['purchased_flag']

In [129]:
df2

,0,1,2,3,4,5,6,7,8,9,...,81,82,83,84,85,86,87,88,89,label
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
611203,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
611204,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
611205,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
611206,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [10]:
df.loc[df['purchase_date'] >= '2021-09-01']

,dow,purchase_date,purchases,purchased_flag,buyer_cluster,open_purchases_daily_30,open_purchases_daily_pct_30,open_purchases_daily_180,open_purchases_daily_pct_180,avg_time_between_open_purchases,...,cars_moved_off_retail_90,cluster_G,cluster_NA,cluster_NG,cluster_O,cluster_P+,cluster_P-,cluster_SC,cluster_Super,buyer_id_masked
602121,3,2021-09-01,0,0,SC,0,0.0,0,0.00,90,...,882,0,0,0,0,0,0,1,0,445
602122,3,2021-09-01,0,0,NG,0,0.0,0,0.00,21,...,0,0,0,1,0,0,0,0,0,482
602123,3,2021-09-01,0,0,NG,0,0.0,1,0.06,90,...,225,0,0,1,0,0,0,0,0,479
602124,3,2021-09-01,0,0,NG,0,0.0,4,0.57,90,...,434,0,0,1,0,0,0,0,0,1395
602125,3,2021-09-01,0,0,NG,0,0.0,0,0.00,31,...,111,0,0,1,0,0,0,0,0,2138
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
611203,4,2021-09-30,0,0,O,0,0.0,5,0.14,4,...,0,0,0,0,1,0,0,0,0,1097
611204,4,2021-09-30,0,0,O,0,0.0,2,0.13,90,...,1518,0,0,0,1,0,0,0,0,2389
611205,4,2021-09-30,0,0,O,0,0.0,2,0.33,1,...,207,0,0,0,1,0,0,0,0,1261
611206,4,2021-09-30,0,0,SC,0,0.0,1,0.17,1,...,394,0,0,0,0,0,0,1,0,2042


In [12]:
train = df2.iloc[:602121,:]

In [13]:
test = df2.iloc[602121:,:]

In [15]:
train.shape

(602121, 91)

In [19]:
df_neg = train[train['label']==0]
df_pos = train[train['label']==1]

In [20]:
df_neg_1 = df_neg.sample(n=len(df_pos) * 2, random_state = 2022, replace = False)
train1 = pd.concat([df_neg_1,df_pos], axis = 0)

In [14]:
test.shape

(9087, 91)

In [21]:
train1.to_csv('lstm_dataset_train.csv', index=False)

In [18]:
test.to_csv('lstm_dataset_test.csv', index=False)

In [22]:
train1.shape

(110061, 91)